In [ ]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from tensorflow import keras
import pandas as pd
from itertools import combinations
from tqdm import tqdm
from keras import backend as K

In [ ]:
one_hot = tf.keras.layers.CategoryEncoding(num_tokens=5, output_mode="one_hot")

def precision(y_true, y_pred):

    y_true = one_hot(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):

    y_true = one_hot(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [ ]:
validation_dir = '../5-7500-a/validation'

test_dir = '../5-7500-a/test'

train_dir = '../5-7500-a/train'

BATCH_SIZE = 32
IMG_SIZE = (224, 224)


train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 5250 files belonging to 5 classes.
Found 1150 files belonging to 5 classes.
Found 1125 files belonging to 5 classes.


In [ ]:
Xception = tf.keras.models.load_model('./xception.h5')
DenseNet121 = tf.keras.models.load_model('./densenet121.h5')
VGG16 = tf.keras.models.load_model('./vgg16.h5')
ResNet50V2 = tf.keras.models.load_model('./resnet_v2.h5')
InceptionResNetV2 = tf.keras.models.load_model('./inception.h5')
MobileNetV2 = tf.keras.models.load_model('./mobilenet.h5')
EfficientNetB0 = tf.keras.models.load_model('./efficientnet.h5')

Xception._name="Xception"
DenseNet121._name="DenseNet121"
VGG16._name="VGG16"
ResNet50V2._name = "ResNet50V2"
InceptionResNetV2._name = "InceptionResNetV2"
MobileNetV2._name = "MobileNetV2"
EfficientNetB0._name = "EfficientNetB0"

In [ ]:
def create_ansamble(models):
    inputs = tf.keras.Input(shape=(224, 224, 3))

    models_outputs = []
    for model in models:
        models_outputs.append(model(inputs))
    if len(models_outputs) > 1:
        outputs = tf.keras.layers.Average()(models_outputs)
    else:
        outputs = models_outputs[0]
    ansamble = keras.Model(inputs=inputs, outputs=outputs)

    base_learning_rate = 0.001
    ansamble.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy', precision, recall, f1])
    return ansamble

In [ ]:
models = {'Xception': Xception, 'DenseNet121': DenseNet121, 'VGG16': VGG16, 'ResNet50V2': ResNet50V2, 'InceptionResNetV2': InceptionResNetV2,
          'MobileNetV2': MobileNetV2, 'EfficientNetB0': EfficientNetB0}

all_combs = []
for i in range(1, len(models)):
    all_combs.extend(list(combinations(models, i)))

ansambles = {}
for comb in all_combs:
    ansambles['-'.join(comb)] = comb

In [ ]:
print("All combinations: {}".format(len(ansambles)))

All combinations: 126


In [ ]:
result = pd.DataFrame({'name':[], 'test_loss': [], 'test_accuracy': [], 'test_precision': [], 'test_recall': [], 'test_f1': [], 'vallidation_accurasy': []})

for ansamble_name, model_names in tqdm(zip(ansambles, ansambles.values())):
    ansamble_models = [models[name] for name in model_names]
    ansamble = create_ansamble(ansamble_models)
    test_loss, test_acc, test_pres, test_recall, test_f1 = ansamble.evaluate(test_dataset)
    _, validation_acc, _, _, _ = ansamble.evaluate(validation_dataset)
    result.loc[len(result)] = [ansamble_name, test_loss, test_acc, test_pres, test_recall, test_f1, validation_acc]

0it [00:00, ?it/s]

36/36 [==============================] - 43s 921ms/step - loss: 0.3785 - accuracy: 0.8617 - precision: 0.8734 - recall: 0.8496 - f1: 0.8612


1it [01:37, 97.52s/it]

36/36 [==============================] - 43s 907ms/step - loss: 0.2046 - accuracy: 0.9270 - precision: 0.9383 - recall: 0.9236 - f1: 0.9308


2it [03:16, 98.33s/it]

...

124it [3:49:21, 122.15s/it]

36/36 [==============================] - 45s 955ms/step - loss: 0.2092 - accuracy: 0.9557 - precision: 0.9624 - recall: 0.9523 - f1: 0.9572


125it [3:51:18, 120.49s/it]

36/36 [==============================] - 45s 968ms/step - loss: 0.1981 - accuracy: 0.9548 - precision: 0.9611 - recall: 0.9488 - f1: 0.9548


126it [3:53:21, 111.12s/it]


In [ ]:
result.to_csv('./results.csv')

In [ ]:
result

,name,test_loss,test_accuracy,test_precision,test_recall,test_f1,vallidation_accurasy
0,Xception,0.249143,0.935111,0.942538,0.931424,0.936828,0.861739
1,DenseNet121,0.306656,0.876444,0.891850,0.870660,0.880900,0.926957
2,VGG16,0.483240,0.864889,0.866336,0.861632,0.863947,0.901739
3,ResNet50V2,0.401022,0.856000,0.862674,0.846875,0.854561,0.900870
4,InceptionResNetV2,0.514581,0.824889,0.837776,0.821181,0.829299,0.906087
...,...,...,...,...,...,...,...
121,Xception-DenseNet121-VGG16-ResNet50V2-MobileNe...,0.197147,0.950222,0.954361,0.934549,0.944258,0.955652
122,Xception-DenseNet121-VGG16-InceptionResNetV2-M...,0.207266,0.951111,0.961038,0.938889,0.949455,0.957391
123,Xception-DenseNet121-ResNet50V2-InceptionResNe...,0.213678,0.926222,0.935260,0.915451,0.924876,0.958261
124,Xception-VGG16-ResNet50V2-InceptionResNetV2-Mo...,0.209793,0.942222,0.954014,0.932292,0.942882,0.955652


In [ ]:
# find the best combination based on validation accuracy
ind = result.vallidation_accurasy.argmax() # find the maximum validation accuracy !
best_model = result.name.loc[ind]
print("The best model is: {}".format(best_model))

The best model is: DenseNet121-InceptionResNetV2-MobileNetV2-EfficientNetB0


In [ ]:
# print test accuracy/loss for this model
print(result.loc[ind])

name                    DenseNet121-InceptionResNetV2-MobileNetV2-Effi...
test_loss                                                        0.218126
test_accuracy                                                        0.92
test_precision                                                   0.928028
test_recall                                                      0.917535
test_f1                                                          0.922672
vallidation_accurasy                                                 0.96
Name: 92, dtype: object


In [ ]:
# create the best ensamble model again
ansamble_models = [models[name] for name in best_model.split("-")]
best_ansamble_model = create_ansamble(ansamble_models)
# save this model
best_ansamble_model.save('model_2.h5')

In [ ]:
# test saved model
from tensorflow.keras.models import load_model
loaded_model = load_model('model_2.h5', custom_objects={"precision": precision, 'recall': recall, 'f1': f1})
loaded_model.summary()

Model: "model_263"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_265 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 DenseNet121 (Functional)       (None, 5)            7042629     ['input_265[0][0]']              
                                                                                                  
 InceptionResNetV2 (Functional)  (None, 5)           54344421    ['input_265[0][0]']              
                                                                                                  
 MobileNetV2 (Functional)       (None, 5)            2264389     ['input_265[0][0]']      